In [ ]:
import pandas as pd
import numpy as np
import panel as pn
from datetime import datetime

pn.extension('tabulator')
pn.extension()

# specific imports
import requests
import re
from IPython.display import Markdown, display

In [ ]:
def printmd(string):
    display(Markdown(string))

In [ ]:
printmd("<b><span style='color:red; font-size: 150%;'>In the next cell, please select and locate your SuAVE CSV file.</span></b>")

In [ ]:
s_files = pn.widgets.FileSelector('~')

s_files

In [ ]:
suave = pd.read_csv(s_files.value[0])

In [ ]:
pn.widgets.Tabulator(suave)

In [ ]:
printmd("<b><span style='color:red; font-size: 150%;'>In the next cell, please select and locate the VV file you downloaded from LimeSurvey.</span></b>")

In [ ]:
v_files = pn.widgets.FileSelector('~')

v_files

In [ ]:
vv = pd.read_csv(v_files.value[0],sep='\t')
old_header = vv.columns
new_header = vv.iloc[0] #grab the first row for the header
vv = vv[1:] #take the data less the header row
vv.columns = new_header

In [ ]:
printmd("<b><span style='color:red; font-size: 150%;'>In the next cell, please select and locate the particpants file you downloaded from LimeSurvey.</span></b>")

In [ ]:
p_files = pn.widgets.FileSelector('~')

p_files

In [ ]:
part = pd.read_csv(p_files.value[0])

In [ ]:
printmd("<b><span style='color:red; font-size: 150%;'>Please input your survey id.</span></b>")
survey_id = input()

In [ ]:
vv['token'] = part['token']
vv['id']=range(1,len(part)+1)
vv['lastpage']=1
vv['startlanguage'] = 'en'
vv['refurl']='https://limesurvey.sdsc.edu/limesurvey/index.php/admin/tokens/sa/browse/surveyid/'+survey_id
vv['submitdate'] = datetime.now().strftime("%m-%d-%y %H:%M:%S")
vv['startdate'] = datetime.now().strftime("%m-%d-%y %H:%M:%S")
vv['datestamp'] = datetime.now().strftime("%m-%d-%y %H:%M:%S")
vv['ipaddr'] = '123.123.123.123'
vv['seed'] = 123456789

# Short Text Questions & Multi Short Text Questions (rerun this section for each short text question)

In [ ]:
printmd("<b><span style='color:red; font-size: 150%;'>For a short text question (works on a multi short text question too), please select the exact column name/s (from the vv file, the table above this cell) corresponding to each subquestion.</span></b>")
vmulti_text = pn.widgets.CrossSelector(name='VV Short Question', options=list(vv.columns))

vmulti_text

In [ ]:
printmd("<b><span style='color:red; font-size: 150%;'>For a short text question (works on a multi short text question too), please select the exact column name/s (from the SuAVE file) corresponding to each subquestion. Make sure the order corresponds to the order above.</span></b>")
smulti_text = pn.widgets.CrossSelector(name='SuAVE Short Question', options=list(suave.columns), definition_order=False)

smulti_text

In [ ]:
for i in range(0,len(vmulti_text.value)):
    vv[vmulti_text.value[i]] = suave[smulti_text.value[i]]

# List Questions (rerun this section for each list question)

In [ ]:
printmd("<b><span style='color:red; font-size: 150%;'>For a list question, please select the exact column name (from the VV file) corresponding to the list question.</span></b>")
listq = pn.widgets.CrossSelector(name='List Question', options=list(vv.columns), definition_order=False)

listq

In [ ]:
printmd("<b><span style='color:red; font-size: 150%;'>For a list question, please select the exact column name (from the SuAVE file) corresponding to the list question.</span></b>")
lista = pn.widgets.CrossSelector(name='List Question', options=list(suave.columns), definition_order=False)

lista

In [ ]:
printmd("<b><span style='color:red; font-size: 150%;'>Are these all the answer options in this question: {}?</span></b>".format(suave[lista.value[0]].unique()))
select = pn.widgets.Select(name='', options=[True, False])

select

In [ ]:
if select.value==False:
    printmd("<b><span style='color:red; font-size: 150%;'>Please input all answer options separated by a comma in the list question.</span></b>")
    options = input()
    options = [x.strip() for x in options.split(',')]
elif select.value==True:
    options = suave[lista.value[0]].unique()

In [ ]:
options.sort()
temp = []
options_dict = {}
for i in range(1,len(options)+1):
    options_dict[options[i-1]] = 'A'+str(i)
for i in suave[lista.value[0]].values:
    temp.append(options_dict[i])
vv[listq.value[0]] = temp

In [ ]:
pn.widgets.Tabulator(vv)

# Multiple Choice Questions (rerun this section for each multiple choice question)

In [ ]:
printmd("<b><span style='color:red; font-size: 150%;'>For a multiple choice question, please select the exact column names (from the VV file) corresponding to the multiple choice question.</span></b>")
multipleq = pn.widgets.CrossSelector(name='Multiple Choice Question', options=list(vv.columns), definition_order=False)

multipleq

In [ ]:
multiple_q = multipleq.value
multiple_q.sort(key=len)

In [ ]:
printmd("<b><span style='color:red; font-size: 150%;'>For a multiple choice question, please select the exact column name (from the SuAVE file) corresponding to the list question.</span></b>")
multiple_s = pn.widgets.CrossSelector(name='List Question', options=list(suave.columns), definition_order=False)

multiple_s

In [ ]:
multiple_a = list(set(sum([i.split('|') for i in suave[multiple_s.value[0]].unique()],[])))
multiple_a.sort()
printmd("<b><span style='color:red; font-size: 150%;'>Are these all the subquestions in this question: {}?</span></b>".format(multiple_a))
selectm = pn.widgets.Select(name='', options=[True, False])
selectm

In [ ]:
if selectm.value==False:
    printmd("<b><span style='color:red; font-size: 150%;'>Please input all subquestions separated by a comma in this multiple choice question.</span></b>")
    multiple_a = input()
    multiple_a = [x.strip() for x in multiple_a.split(',')]

In [ ]:
multiple_dict = {}
for i in range(0,len(multiple_a)):
    multiple_dict[multiple_a[i]] = 'Keywords_A'+str(i+1)
for i in range(0,len(multiple_a)):
    keywords = suave.iloc[[i]][multiple_s.value[0]].values[0].split('|')
    for j in keywords:
        vv.loc[vv.index[i],multiple_dict[j]] = 'Y'

# Image Question 

In [ ]:
printmd("<b><span style='color:red; font-size: 150%;'>For the photo question, please select the exact column name (from the VV file) corresponding to the photo question. The column name should be the name of the question only and not the one with filecount.</span></b>")
photoq = pn.widgets.CrossSelector(name='Photo Question', options=list(vv.columns), definition_order=False)
photoq

In [ ]:
vv[photoq.value[0]+'_filecount'] = 1
temp = '[{lbrace} "title":"","comment":"","size":"817.6318359","name":"","filename":"","ext":"png" }]'.split(":")
for j in range(0,len(vv)):
    temp = '[{lbrace} "title":"","comment":"","size":"817.6318359","name":"","filename":"","ext":"png" }]'.split(":")
    img = ''
    for i in range(0,len(temp)):
        if i == 3:
            img = img+temp[i]+":"+"\""+suave['#img'][j]+"\""
        elif i == 4:
            img = img+temp[i][2:]+":"+"\""+suave['#img'][j]+"\""
        elif i == 5:
            img = img+temp[i][2:]+":"
        elif i != 6:
            img = img+temp[i]+":"
        elif i == 6:
            img = img + temp[i]
    vv.loc[vv.index[j],photoq.value[0]] = img

# Review your VV file by running the cell below to check that your columns are correctly populated.

In [ ]:
pn.widgets.Tabulator(vv)

# When you finish populating all your columns, run the cell below to get the updated VV file.

In [ ]:
vv.loc[-1] = vv.columns.values
vv.sort_index(inplace=True)
vv.reset_index(drop=True, inplace=True)

col_dict = {}
for i in range(0,len(old_header)):
    col_dict[new_header[i]] = old_header[i]
vv.rename(columns=col_dict, inplace=True)
vv.to_csv('updated'+survey_id+".csv",index = False)